# Part 03: Merge data and prep to find discontinuities
michael babb  
2025 09 22

In [63]:
# standard
import os

In [64]:
# external
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point

In [65]:
# custom
import run_constants as rc
from geo_data_io.df_operations import hey_what_is_na
from geo_data_io.fc_df_spatial import write_gdf
from utils import get_sort_order

# load the working seattle street network data

In [66]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S03_SND_WORKING_IN_FILE_NAME)

In [67]:
gdf = gpd.read_file(filename = fpn)

In [68]:
gdf.shape

(25771, 39)

In [69]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_sector',
       'ord_stname_unique', 'geometry'],
      dtype='object')

# load the manually created street group data
These data were created in Step 6, but we'll bring them into the Step 3 workflow.

In [70]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S03_STREET_GROUP_IN_FILE_NAME)

In [71]:
index_cols = ['sort_order', 'ord_street_name', 'ord_street_type', 'city_sector']
sg_df = pd.read_excel(io = fpn, index_col=[0, 1, 2, 3])
sg_df = sg_df.reset_index()

In [72]:
sg_df.head()

,sort_order,ord_street_name,ord_street_type,city_sector,group_id,progress,count
0,001,1ST,AVE,CNTR,2,0.000824,1
1,001,1ST,AVE,N,2,0.001647,2
2,001,1ST,AVE,NE,2,0.002471,3
3,001,1ST,AVE,NW,1,0.003295,4
4,001,1ST,AVE,S,2,0.004119,5


In [73]:
drop_cols = ['sort_order', 'progress', 'count']
sg_df = sg_df.drop(labels = drop_cols, axis = 1)

In [74]:
hey_what_is_na(sg_df)

In [75]:
sg_df.loc[
        (sg_df['ord_street_name']=='55TH') &
        (sg_df['ord_street_type'] == 'PL'), :]

,ord_street_name,ord_street_type,city_sector,group_id
452,55TH,PL,NE,1
453,55TH,PL,NW,2


In [76]:
sg_df.head()

,ord_street_name,ord_street_type,city_sector,group_id
0,1ST,AVE,CNTR,2
1,1ST,AVE,N,2
2,1ST,AVE,NE,2
3,1ST,AVE,NW,1
4,1ST,AVE,S,2


In [77]:
gdf = pd.merge(left = gdf, right = sg_df, how = 'left', indicator=True)
gdf['group_id'] = gdf['group_id'].fillna(1).astype(int)

In [78]:
gdf.shape

(25771, 41)

In [79]:
gdf['_merge'].value_counts()

_merge
both          19031
left_only      6740
right_only        0
Name: count, dtype: int64

In [80]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_sector',
       'ord_stname_unique', 'geometry', 'group_id', '_merge'],
      dtype='object')

In [81]:
hey_what_is_na(gdf)

unitid
False    25715
True        56
Name: count, dtype: int64
unitid2
False    25715
True        56
Name: count, dtype: int64


# Manually update groups of streets
In Northeast Seattle, there is a 55TH PL NE and NE 55TH PL.  
These are different streets, and not one street with a discontinuity.  
NE 55TH PL runs parallel to NE 55TH ST.  
55TH PL NE is curved, beginning at the northern end of 55TH AVE NE.

In [82]:
tgdf = gdf.loc[
        (gdf['ord_street_name']=='55TH') &
        (gdf['ord_street_type'] == 'PL'), :]

In [83]:
tgdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,sndseg_update,compkey,comptype,unitid,unitid2,city_sector,ord_stname_unique,geometry,group_id,_merge
3414,5551,5553,7609,1,1,456,0,0,1,1,...,2007-04-22 00:00:00+00:00,7734,68,04225,0750,NE,55TH PL NE,"LINESTRING (-122.26899 47.68334, -122.26899 47...",1,both
5390,6828,6913,9865,5,1,3461,0,1,1,1,...,2005-05-04 00:00:00+00:00,18443,68,14545,0550,NW,NW 55TH PL,"LINESTRING (-122.36082 47.66943, -122.36091 47...",2,both
8152,6959,6960,13081,1,1,1864,0,0,1,1,...,2005-05-04 00:00:00+00:00,17020,68,13370,0170,NE,NE 55TH PL,"LINESTRING (-122.30887 47.66894, -122.30709 47...",1,both


In [84]:
# manually assign to a different group
gdf.loc[gdf['ord_stname_concat'] == 'NE 55TH PL', 'group_id'] = 3

In [85]:
# check for NA
hey_what_is_na(gdf)

unitid
False    25715
True        56
Name: count, dtype: int64
unitid2
False    25715
True        56
Name: count, dtype: int64


In [86]:
# that's okay

In [87]:
# drop the _merge
gdf = gdf.drop(labels = ["_merge"], axis = 1)

# create an id across street groups

In [88]:
col_names = ['ord_street_name', 'ord_street_type',  'ord_stname_concat', 'ord_stname_unique', 'city_sector', 'group_id']
id_df = gdf[col_names].drop_duplicates().reset_index(drop=True)

In [89]:
id_df['ord_stname_type'] = id_df['ord_street_name'] + ' ' + id_df['ord_street_type']

In [90]:
id_df['sort_order'] = id_df['ord_street_name'].map(get_sort_order)

In [91]:
id_df = id_df.sort_values(by = ['sort_order', 'ord_street_type', 'city_sector']).reset_index(drop = True)

In [92]:
id_df.head()

,ord_street_name,ord_street_type,ord_stname_concat,ord_stname_unique,city_sector,group_id,ord_stname_type,sort_order
0,1ST,AVE,1ST AVE,1ST AVE CNTR,CNTR,2,1ST AVE,001
1,1ST,AVE,1ST AVE N,1ST AVE N,N,2,1ST AVE,001
2,1ST,AVE,1ST AVE NE,1ST AVE NE,NE,2,1ST AVE,001
3,1ST,AVE,1ST AVE NW,1ST AVE NW,NW,1,1ST AVE,001
4,1ST,AVE,1ST AVE S,1ST AVE S,S,2,1ST AVE,001


In [93]:
hey_what_is_na(id_df)

In [94]:
id_df.shape

(2457, 8)

In [95]:
# check 55TH PL NE and NE 55TH PL
id_df.loc[
        (id_df['ord_street_name']=='55TH') &
        (id_df['ord_street_type'] == 'PL'), :]

,ord_street_name,ord_street_type,ord_stname_concat,ord_stname_unique,city_sector,group_id,ord_stname_type,sort_order
500,55TH,PL,55TH PL NE,55TH PL NE,NE,1,55TH PL,055
501,55TH,PL,NE 55TH PL,NE 55TH PL,NE,3,55TH PL,055
502,55TH,PL,NW 55TH PL,NW 55TH PL,NW,2,55TH PL,055


# Aggregate by groups

In [96]:
col_names = ['sort_order', 'ord_street_name', 'ord_street_type', 'ord_stname_type', 'group_id', 'city_sector']
agg_id_df = id_df[col_names].groupby(col_names[:-1]).agg(
    city_sector_group = ('city_sector', lambda x: '_'.join(sorted(set(x)))),
    n_groups = ('city_sector', lambda x: len(set(x)))).reset_index()

In [97]:
hey_what_is_na(agg_id_df)

In [98]:
agg_id_df.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_sector_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [99]:
agg_id_df.loc[
        (agg_id_df['ord_street_name']=='55TH') &
        (agg_id_df['ord_street_type'] == 'PL'), :]

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_sector_group,n_groups
245,055,55TH,PL,55TH PL,1,NE,1
246,055,55TH,PL,55TH PL,2,NW,1
247,055,55TH,PL,55TH PL,3,NE,1


# import the revised city_sector_group

In [100]:
fpn = os.path.join(rc.INPUT_FILE_PATH, 'city_sector_check_working.xlsx')

In [101]:
cpg_df = pd.read_excel(io = fpn)

In [102]:
cpg_df.head()

,city_sector_group,ord_street_type,n_groups,city_sector_group_revised
0,CNTR,AVE,1,NaN
1,CNTR,BR,1,NaN
2,CNTR,CT,1,NaN
3,CNTR,DR,1,NaN
4,CNTR,PL,1,NaN


In [103]:
hey_what_is_na(cpg_df)

city_sector_group_revised
True     87
False    39
Name: count, dtype: int64


In [104]:
cpg_df.loc[cpg_df['ord_street_type'] == 'PL', :]

,city_sector_group,ord_street_type,n_groups,city_sector_group_revised
4,CNTR,PL,1,NaN
33,E,PL,1,NaN
48,N,PL,1,NaN
59,NE,PL,1,NaN
72,NW,PL,1,NaN
90,S,PL,1,NaN
102,SW,PL,1,NaN
121,W,PL,1,NaN


In [105]:
# build a row selector. Less code!
row_select = cpg_df['city_sector_group_revised'].isna()

In [106]:
# update where null
cpg_df.loc[row_select, 'city_sector_group_revised'] = cpg_df.loc[row_select, 'city_sector_group']

In [107]:
cpg_df.head()

,city_sector_group,ord_street_type,n_groups,city_sector_group_revised
0,CNTR,AVE,1,CNTR
1,CNTR,BR,1,CNTR
2,CNTR,CT,1,CNTR
3,CNTR,DR,1,CNTR
4,CNTR,PL,1,CNTR


In [108]:
hey_what_is_na(df = cpg_df)

In [109]:
hey_what_is_na(df = agg_id_df)

In [110]:
display(cpg_df.loc[cpg_df['ord_street_type'] == 'PL', ])

,city_sector_group,ord_street_type,n_groups,city_sector_group_revised
4,CNTR,PL,1,CNTR
33,E,PL,1,E
48,N,PL,1,N
59,NE,PL,1,NE
72,NW,PL,1,NW
90,S,PL,1,S
102,SW,PL,1,SW
121,W,PL,1,W


In [111]:
test_join = pd.merge(left = agg_id_df, right = cpg_df, how = 'outer', indicator=True)

In [112]:
test_join['_merge'].value_counts()

_merge
both          1817
left_only        0
right_only       0
Name: count, dtype: int64

In [113]:
agg_id_df.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_sector_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [114]:
agg_id_df = test_join.drop(labels = ['city_sector_group', '_merge'], axis = 1)

In [115]:
hey_what_is_na(df = agg_id_df)    

In [116]:
test_join = pd.merge(left = gdf, right = agg_id_df, how = 'outer', indicator=True)

In [117]:
test_join['_merge'].value_counts()

_merge
both          25771
left_only         0
right_only        0
Name: count, dtype: int64

In [118]:
test_join['city_sector_group'] = test_join['city_sector_group_revised']

In [119]:
gdf = test_join.drop(labels = ['_merge', 'city_sector_group_revised', 'n_groups'], axis = 1)

In [120]:
gdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,comptype,unitid,unitid2,city_sector,ord_stname_unique,geometry,group_id,sort_order,ord_stname_type,city_sector_group
0,3836,3893,4787,5,1,1955,0,1,1,1,...,68,14050,0040,NW,NW 100TH PL,"LINESTRING (-122.36206 47.7035, -122.36342 47....",1,100,100TH PL,NW
1,3893,3906,4802,5,1,1955,0,1,1,1,...,68,14050,0043,NW,NW 100TH PL,"LINESTRING (-122.36342 47.70275, -122.36386 47...",1,100,100TH PL,NW
2,3906,3973,4864,5,1,1955,0,1,1,1,...,68,14050,0070,NW,NW 100TH PL,"LINESTRING (-122.36386 47.70251, -122.36535 47...",1,100,100TH PL,NW
3,48444,48443,49871,1,0,3767,0,0,1,1,...,0,0,0,SW,SW 100TH PL,"LINESTRING (-122.3403 47.51334, -122.34036 47....",2,100,100TH PL,SW
4,3943,3939,4836,1,1,1956,0,0,1,1,...,68,14055,0240,NW,NW 100TH ST,"LINESTRING (-122.38757 47.70155, -122.38762 47...",1,100,100TH ST,NW_N_NE


In [121]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_sector',
       'ord_stname_unique', 'geometry', 'group_id', 'sort_order',
       'ord_stname_type', 'city_sector_group'],
      dtype='object')

In [122]:
gdf['ord_stname_type_group'] = gdf['ord_stname_type'] + ' ' + gdf['city_sector_group']

In [123]:
# export to disk
write_gdf(gdf = gdf, output_file_path=rc.OUTPUT_FILE_PATH, output_file_name=rc.S03_CLEANED_STREET_GROUP_OUT_FILE_NAME)

In [124]:
hey_what_is_na(df = gdf)

unitid
False    25715
True        56
Name: count, dtype: int64
unitid2
False    25715
True        56
Name: count, dtype: int64
